<a href="https://colab.research.google.com/github/RemyLpr/defi_ia/blob/main/data/req_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import des modules et des données

In [194]:
import urllib.parse
import requests
import numpy as np
import pandas as pd
import random
import string
import csv
from IPython.display import display
import io
from google.colab import files

In [111]:
df_test = pd.read_csv("test_set.csv")
coeffs_date = pd.read_csv("coeffs_date.csv")
coeffs_computer = pd.read_csv("coeffs_computer.csv")

# Initialisation session API

In [205]:
domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)
user_id = 'e68dad03-0d1a-4038-8766-382b25fe4252'

# Requêtes restantes

In [211]:
r = requests.get(path(f"remaining-requests/{user_id}"))
print("Nombre de requêtes restantes :", r.json())

Nombre de requêtes restantes : 881


# Génération des paramètres

In [81]:
def params_generator(df_test, coeffs_date, coeffs_computer):
  # Pour city, répartition uniforme
  cities = df_test["city"].unique().tolist() # array par défaut
  city = random.choice(cities) # ville choisie
  # Pour date, suivre répartition fichier coeffs_date
  date = random.choices(coeffs_date["date"], coeffs_date["coeff"], k = 1)[0] # liste renvoyée, on prend premier terme car k = 1
  # Pour language, répartition uniforme également (choix par défaut)
  languages = df_test["language"].unique().tolist()
  language = random.choice(languages) # langue choisie
  # Pour mobile, répartition suivant le fichier coeffs_computer
  mobile_prob = coeffs_computer["computer"].tolist()[int(date)] # on sélectionne la proba que mobile = 0 correspondant à la date sélectionnée
  mobile_prob = [mobile_prob, 1 - mobile_prob]
  mobile = random.choices([0, 1], mobile_prob, k = 1)[0]
  # Pour l'instant on utilie un avatar par requête (temporaire)
  avatar = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6)) # génération d'un pseudo aléatoire
  # Définition du dictionnaire des paramètres
  params = {"city" : city, "date" : date, "language" : language, "mobile" : mobile, "avatar_name" : avatar}
  return params

In [114]:
params = params_generator(df_test, coeffs_date, coeffs_computer)
params

{'city': 'amsterdam',
 'date': 22,
 'language': 'austrian',
 'mobile': 1,
 'avatar_name': '13B2YT'}

# Requêtage de l'API

**IMPORTANT** <br>
Avant de requêter il est nécessaire de pull la dernière version du fichier train_set.csv depuis le git et ensuite l'importer manuellement dans Colab. Une fois les requêtes effectuées il faut sauvegarder le fichier à l'emplacement local du git et push. Mettre un commentaire suivant le format : <br> prénom + nb_req total + date + heure

In [212]:
nb_req = int(input("Entrez le nombre de requêtes à exécuter "))
old_df = pd.read_csv("train_set.csv")
df = old_df.copy()
req_count = int(old_df["order_requests"].tolist()[-1]) # on récupère la dernière colonne (order_requests) de la dernière ligne
for i in range(1, nb_req + 1):
  params = params_generator(df_test, coeffs_date, coeffs_computer)
  requests.post(path(f'avatars/{user_id}/{params["avatar_name"]}')) # on doit indiquer à l'API qu'on crée ce nouvel avatar
  r = requests.get(path(f"pricing/{user_id}"), params = params)
  result = r.json() # résultat de la requête au format json
  query_res = pd.DataFrame(result['prices'])
  params_df = pd.DataFrame([result['request']]*len(query_res))
  new_df = pd.concat([query_res, params_df], axis=1)
  new_df["avatar_name"] = params["avatar_name"] # ajout du nom de l'avatar
  new_df["order_requests"] = req_count + i # ajout de la colonne order_requests comme dans le test_set
  # on ajoute les données extraites à notre dataset
  df = pd.concat([df, new_df])
  df.to_csv("train_set.csv", index=False) # pour pas écrire la colonne Unnamed : 0
print("Requêtage terminé !")

Entrez le nombre de requêtes à exécuter 500
Requêtage terminé !


In [220]:
data = pd.read_csv("train_set.csv")
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.to_csv("train_set.csv", index=False)

# Requêtage et enregistrement des résultats

In [ ]:
def save_data(result):

    query_res = pd.DataFrame(result['prices'])
    params = pd.DataFrame([result['request']]*len(query_res))
    df = pd.concat([query_res, params], axis=1)
    display(df)
    
    with open('train_set.csv', 'a') as f:
             df.to_csv(f, header=f.tell()==0, index=False)

In [ ]:
# params = {
#     "avatar_name": "nono10",
#     "language": "luxembourgish",
#     "city": "paris",
#     "date": 44,
#     "mobile": 1,
#     }
# r = requests.get(path(f"pricing/{user_id}"), params=params)

In [ ]:
# ATTENTION : n'exécuter qu'une et unique fois pour chaque requête sinon doublage dans les données !!
# save_df(r.json())

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,254,144,140,paris,44,luxembourgish,1,137635
1,722,145,141,paris,44,luxembourgish,1,137635
2,308,168,195,paris,44,luxembourgish,1,137635
3,782,243,276,paris,44,luxembourgish,1,137635
4,866,240,280,paris,44,luxembourgish,1,137635
...,...,...,...,...,...,...,...,...
162,595,141,140,paris,44,luxembourgish,1,137635
163,359,106,110,paris,44,luxembourgish,1,137635
164,199,171,197,paris,44,luxembourgish,1,137635
165,159,165,195,paris,44,luxembourgish,1,137635


## Essais

In [ ]:
# paris - luxembourgish - mobile = 1, avatar_name = nono10

# for req in range(43, -1, -1):
#     params = {
#     "avatar_name": "nono10",
#     "language": "luxembourgish",
#     "city": "paris",
#     "date": req,
#     "mobile": 1,
#     }
#     r = requests.get(path(f"pricing/{user_id}"), params=params)
    
#     save_df(r.json())

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,617,247,257,paris,43,luxembourgish,1,137635
1,130,166,184,paris,43,luxembourgish,1,137635
2,392,149,130,paris,43,luxembourgish,1,137635
3,229,174,186,paris,43,luxembourgish,1,137635
4,742,237,267,paris,43,luxembourgish,1,137635
...,...,...,...,...,...,...,...,...
162,825,147,132,paris,43,luxembourgish,1,137635
163,436,247,267,paris,43,luxembourgish,1,137635
164,782,254,256,paris,43,luxembourgish,1,137635
165,14,143,104,paris,43,luxembourgish,1,137635


,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,579,80,50,paris,42,luxembourgish,1,137635
1,564,92,48,paris,42,luxembourgish,1,137635
2,614,83,49,paris,42,luxembourgish,1,137635
3,69,261,256,paris,42,luxembourgish,1,137635
4,379,258,251,paris,42,luxembourgish,1,137635
...,...,...,...,...,...,...,...,...
162,700,184,179,paris,42,luxembourgish,1,137635
163,477,80,124,paris,42,luxembourgish,1,137635
164,339,179,176,paris,42,luxembourgish,1,137635
165,577,152,122,paris,42,luxembourgish,1,137635


,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,748,184,167,paris,41,luxembourgish,1,137635
1,447,111,90,paris,41,luxembourgish,1,137635
2,842,267,216,paris,41,luxembourgish,1,137635
3,299,106,118,paris,41,luxembourgish,1,137635
4,767,154,110,paris,41,luxembourgish,1,137635
...,...,...,...,...,...,...,...,...
162,602,111,48,paris,41,luxembourgish,1,137635
163,182,159,118,paris,41,luxembourgish,1,137635
164,301,109,93,paris,41,luxembourgish,1,137635
165,306,188,156,paris,41,luxembourgish,1,137635


,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,741,156,107,paris,40,luxembourgish,1,137635
1,580,117,84,paris,40,luxembourgish,1,137635
2,868,179,157,paris,40,luxembourgish,1,137635
3,866,263,225,paris,40,luxembourgish,1,137635
4,656,111,89,paris,40,luxembourgish,1,137635
...,...,...,...,...,...,...,...,...
153,576,195,146,paris,40,luxembourgish,1,137635
154,24,256,233,paris,40,luxembourgish,1,137635
155,612,179,155,paris,40,luxembourgish,1,137635
156,492,115,88,paris,40,luxembourgish,1,137635


,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,631,85,41,paris,39,luxembourgish,1,137635
1,534,263,218,paris,39,luxembourgish,1,137635
2,314,112,79,paris,39,luxembourgish,1,137635
3,182,160,109,paris,39,luxembourgish,1,137635
4,842,280,186,paris,39,luxembourgish,1,137635
...,...,...,...,...,...,...,...,...
153,634,86,41,paris,39,luxembourgish,1,137635
154,467,256,198,paris,39,luxembourgish,1,137635
155,274,157,106,paris,39,luxembourgish,1,137635
156,926,181,144,paris,39,luxembourgish,1,137635


...

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,986,337,3,paris,4,luxembourgish,1,137635
1,975,174,6,paris,4,luxembourgish,1,137635
2,937,295,3,paris,4,luxembourgish,1,137635
3,910,92,0,paris,4,luxembourgish,1,137635
4,21,90,0,paris,4,luxembourgish,1,137635
...,...,...,...,...,...,...,...,...
150,176,90,0,paris,4,luxembourgish,1,137635
151,130,331,3,paris,4,luxembourgish,1,137635
152,992,87,0,paris,4,luxembourgish,1,137635
153,225,200,2,paris,4,luxembourgish,1,137635


,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,238,167,0,paris,3,luxembourgish,1,137635
1,776,167,0,paris,3,luxembourgish,1,137635
2,577,197,0,paris,3,luxembourgish,1,137635
3,656,157,1,paris,3,luxembourgish,1,137635
4,338,206,15,paris,3,luxembourgish,1,137635
...,...,...,...,...,...,...,...,...
150,121,87,0,paris,3,luxembourgish,1,137635
151,986,337,2,paris,3,luxembourgish,1,137635
152,222,357,4,paris,3,luxembourgish,1,137635
153,484,145,2,paris,3,luxembourgish,1,137635


,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,290,184,2,paris,2,luxembourgish,1,137635
1,817,317,1,paris,2,luxembourgish,1,137635
2,631,87,0,paris,2,luxembourgish,1,137635
3,306,238,3,paris,2,luxembourgish,1,137635
4,640,202,0,paris,2,luxembourgish,1,137635
...,...,...,...,...,...,...,...,...
150,229,214,3,paris,2,luxembourgish,1,137635
151,123,163,1,paris,2,luxembourgish,1,137635
152,558,107,0,paris,2,luxembourgish,1,137635
153,723,90,0,paris,2,luxembourgish,1,137635


,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,314,123,0,paris,1,luxembourgish,1,137635
1,735,203,0,paris,1,luxembourgish,1,137635
2,342,126,0,paris,1,luxembourgish,1,137635
3,301,128,1,paris,1,luxembourgish,1,137635
4,614,92,0,paris,1,luxembourgish,1,137635
...,...,...,...,...,...,...,...,...
150,397,448,1,paris,1,luxembourgish,1,137635
151,29,334,1,paris,1,luxembourgish,1,137635
152,576,238,0,paris,1,luxembourgish,1,137635
153,612,219,1,paris,1,luxembourgish,1,137635


,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,884,233,0,paris,0,luxembourgish,1,137635
1,584,484,0,paris,0,luxembourgish,1,137635
2,645,169,0,paris,0,luxembourgish,1,137635
3,182,170,0,paris,0,luxembourgish,1,137635
4,290,200,0,paris,0,luxembourgish,1,137635
...,...,...,...,...,...,...,...,...
150,948,312,0,paris,0,luxembourgish,1,137635
151,647,150,0,paris,0,luxembourgish,1,137635
152,782,388,0,paris,0,luxembourgish,1,137635
153,631,87,0,paris,0,luxembourgish,1,137635


In [ ]:
req = pd.read_csv('train_set.csv')
req

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,254,144,140,paris,44,luxembourgish,1,137635
1,722,145,141,paris,44,luxembourgish,1,137635
2,308,168,195,paris,44,luxembourgish,1,137635
3,782,243,276,paris,44,luxembourgish,1,137635
4,866,240,280,paris,44,luxembourgish,1,137635
...,...,...,...,...,...,...,...,...
7236,948,312,0,paris,0,luxembourgish,1,137635
7237,647,150,0,paris,0,luxembourgish,1,137635
7238,782,388,0,paris,0,luxembourgish,1,137635
7239,631,87,0,paris,0,luxembourgish,1,137635
